In [1]:
### This cell imports the required libraries, the selected dataset, outputs the dataset size, and defines the necessary functions for searching it.
#Import libraries
import numpy as np
import pickle

#Load data (binarised)
with open('./WebData_[5, 5, 5, 2, 100].pickle','rb') as file: #...ensure this filepath is correct for the dataset one wishes to search
    Data = pickle.load(file)

#Count number of webs
web_count = 0
for wc in Data:
    for sc in wc: web_count += len(sc)
print(f'Number of webs: {web_count}')
del(file,wc,sc)

#Define function to search database for an input web, returning its position in the database
def ReturnWebClass(web):
    for wc in range(len(Data)):
        for sc in range(len(Data[wc])):
            for w in range(len(Data[wc][sc])):
                if np.array_equal(Data[wc][sc][w],web): 
                    return (wc,sc,w)
    return 'web not present'

#Define a function to order the web's legs in a consistent manner (anticlockwise from the -p axis) --> more efficient data storage and searching
def anticlockwise_websort(web):
    angles = [np.arctan2(web[1,0],web[0,0]),np.arctan2(web[1,1],web[0,1]),np.arctan2(web[1,2],web[0,2])] 
    return np.vstack(list(zip(*sorted(zip(angles,web.transpose()),key=lambda x: x[0])))[1]).transpose()


Number of webs: 1002396


In [2]:
### (1) Return the invariants of a web (i.e. for the full weak class)
#Define input web
input_web = [[-7, 11, -1], [-3,  4,  1], [ 3,  2,  1]]  #...input format [[p1,p2,p3],[q1,q2,q3],[m1,m2,m3]]
#Convert web to an array, and sort its legs
input_web = anticlockwise_websort(np.array(input_web,dtype='int'))

#Check if the web conserves (p,q) charge
if input_web[0][0]*input_web[2][0] + input_web[0][1]*input_web[2][1] + input_web[0][2]*input_web[2][2] != 0 or input_web[1][0]*input_web[2][0] + input_web[1][1]*input_web[2][1] + input_web[1][2]*input_web[2][2] != 0:
    print('Error: (p,q) charge not conserved for the web')
#Check the web is displayed with all (p,q) coprime --> correct it to if not
for leg_idx in range(3):
    if abs(np.gcd(input_web[0][leg_idx],input_web[1][leg_idx])) != 1:
        print('Error: Leg '+str(leg_idx+1)+' (p,q) values not coprime, correcting web...')
        factor = abs(np.gcd(input_web[0][leg_idx],input_web[1][leg_idx]))
        input_web[2][leg_idx] *= factor
        input_web[0][leg_idx] = int(input_web[0][leg_idx]/factor)
        input_web[1][leg_idx] = int(input_web[1][leg_idx]/factor)
        
#Compute the web rank
I = abs(input_web[0][0]*input_web[2][0]*input_web[1][1]*input_web[2][1]-input_web[0][1]*input_web[2][1]*input_web[1][0]*input_web[2][0] + input_web[0][0]*input_web[2][0]*input_web[1][2]*input_web[2][2]-input_web[0][2]*input_web[2][2]*input_web[1][0]*input_web[2][0] + input_web[0][1]*input_web[2][1]*input_web[1][2]*input_web[2][2]-input_web[0][2]*input_web[2][2]*input_web[1][1]*input_web[2][1]) - (input_web[2][0]**2 + input_web[2][1]**2 + input_web[2][2]**2)
web_rank = (I+2)/2
print(f'Web rank: {web_rank}')

#Compute the web total monodromy trace
M1 = np.array([[1+input_web[0][0]*input_web[1][0],-input_web[0][0]**2],[input_web[1][0]**2,1-input_web[0][0]*input_web[1][0]]]) #...monodormy for leg 1
M2 = np.array([[1+input_web[0][1]*input_web[1][1],-input_web[0][1]**2],[input_web[1][1]**2,1-input_web[0][1]*input_web[1][1]]]) #...monodormy for leg 2
M3 = np.array([[1+input_web[0][2]*input_web[1][2],-input_web[0][2]**2],[input_web[1][2]**2,1-input_web[0][2]*input_web[1][2]]]) #...monodormy for leg 3
web_monodromy_trace = np.matrix.trace(np.matmul(M3,np.matmul(M2,M1)))
print(f'Web total monodromy trace: {web_monodromy_trace}')

#Compute the web asymptotic charge
web_asymptotic_charge = abs(np.gcd.reduce([input_web[0][0]*input_web[1][1] - input_web[0][1]*input_web[1][0],input_web[0][0]*input_web[1][2] - input_web[0][2]*input_web[1][0],input_web[0][1]*input_web[1][2] - input_web[0][2]*input_web[1][1]]))
print(f'Web asymptotic charge: {web_asymptotic_charge}')


Web rank: 9.0
Web total monodromy trace: 402
Web asymptotic charge: 5


In [3]:
### (2) Return the class of equivalent webs for an input web
#Define input web
input_web = [[-7, 11, -1], [-3,  4,  1], [ 3,  2,  1]]  #...input format [[p1,p2,p3],[q1,q2,q3],[m1,m2,m3]]
#Convert web to an array, and sort its legs
input_web = anticlockwise_websort(np.array(input_web,dtype='int'))

#Identify the web's position in the database
web_position = ReturnWebClass(input_web)
print(f'The indices to denote the web position in the database are: {web_position}\t--> (weak class index, strong class index, strong class position)',end='\n\n#####################\n\nSample strong eqivalent webs:\n')

#Print a sample of webs strongly equivalent to the input web, change number to return with the 'size' option
for web in np.random.choice(range(len(Data[web_position[0]][web_position[1]])), size=5, replace=False):
    print(Data[web_position[0]][web_position[1]][web],end='\n\n')


The indices to denote the web position in the database are: (2, 0, 55)	--> (weak class index, strong class index, strong class position)

#####################

Sample strong eqivalent webs:
[[  3   3 -31]
 [ -1   4   7]
 [ 29   2   3]]

[[-1  3  2]
 [-5  5 15]
 [ 7  1  2]]

[[ 45  15  -5]
 [-29  -2   3]
 [  7   2  69]]

[[ 14   3 -49]
 [-25  -5  85]
 [  2   7   1]]

[[  3 -19  -1]
 [-11  73   2]
 [  7   1   2]]



In [5]:
### (3) Return whether 2 inputs webs are strong/weak equivalent
#Define input webs, format: [[p1,p2,p3],[q1,q2,q3],[m1,m2,m3]]
input_web_1 = [[ 79,  14, -73], [-18,  -3,  16], [  1,  10,   3]]
input_web_2 = [[ -1,  -7,  19], [ -4, -53,  91], [ 74,   3,   5]]
#Convert webs to arrays, and sort their legs
input_web_1, input_web_2 = anticlockwise_websort(np.array(input_web_1,dtype='int')), anticlockwise_websort(np.array(input_web_2,dtype='int'))

#Find the webs in the database
web1_position = ReturnWebClass(input_web_1)
web2_position = ReturnWebClass(input_web_2)

#Output the equivalences
if isinstance(web1_position,str) or isinstance(web2_position,str): 
    print('...cannot confirm equivalences as webs outside used dataset')
else:
    if web1_position[:2] == web2_position[:2]:
        print('Webs STRONG equivalent (and hence weak equivalent also)')
    elif web1_position[0] == web2_position[0]:
        print('Webs WEAK equivalent (but not strong equivalent)')
    else: 
        print('Webs NOT strong or weak equivalent')

print(f'\nData indexes: web 1 --> {web1_position}, web 2 --> {web2_position}')


Webs WEAK equivalent (but not strong equivalent)

Data indexes: web 1 --> (113, 0, 1444), web 2 --> (113, 1, 965)
